In [2]:
import pandas as pd
import numpy as np
import datetime
import holidays


df = pd.read_csv('../data/train_data.csv')
df['datetime'] = pd.to_datetime(df['datetime'])
df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,datetime
0,7475,CA-2014-167199,1/6/2014,1/10/2014,Standard Class,ME-17320,Maria Etezadi,Home Office,United States,Henderson,...,South,FUR-CH-10004063,Furniture,Chairs,Global Deluxe High-Back Manager's Chair,2573.820,9,0.0,746.4078,2014-01-06
1,7661,CA-2014-105417,1/7/2014,1/12/2014,Standard Class,VS-21820,Vivek Sundaresam,Consumer,United States,Huntsville,...,Central,FUR-FU-10004864,Furniture,Furnishings,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",76.728,3,0.6,-53.7096,2014-01-07
2,867,CA-2014-149020,1/10/2014,1/15/2014,Standard Class,AJ-10780,Anthony Jacobs,Corporate,United States,Springfield,...,South,FUR-FU-10000965,Furniture,Furnishings,"Howard Miller 11-1/2"" Diameter Ridgewood Wall ...",51.940,1,0.0,21.2954,2014-01-10
3,717,CA-2014-130092,1/11/2014,1/14/2014,First Class,SV-20365,Seth Vernon,Consumer,United States,Dover,...,East,FUR-FU-10000010,Furniture,Furnishings,"DAX Value U-Channel Document Frames, Easel Back",9.940,2,0.0,3.0814,2014-01-11
4,2979,CA-2014-109232,1/13/2014,1/16/2014,Second Class,ND-18370,Natalie DeCherney,Consumer,United States,Mount Pleasant,...,South,FUR-CH-10000422,Furniture,Chairs,Global Highback Leather Tilter in Burgundy,545.940,6,0.0,87.3504,2014-01-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1691,2092,CA-2017-166142,7/15/2017,7/19/2017,Standard Class,MM-17260,Magdelene Morse,Consumer,United States,Wilmington,...,East,FUR-TA-10004607,Furniture,Tables,Hon 2111 Invitation Series Straight Table,310.443,3,0.3,-48.7839,2017-07-15
1692,5501,CA-2017-164378,7/15/2017,7/18/2017,Second Class,MM-18055,Michelle Moray,Consumer,United States,New York City,...,East,FUR-CH-10002084,Furniture,Chairs,Hon Mobius Operator's Chair,664.146,6,0.1,88.5528,2017-07-15
1693,7691,US-2017-128951,7/15/2017,7/17/2017,First Class,RS-19420,Ricardo Sperren,Corporate,United States,Suffolk,...,South,FUR-TA-10004575,Furniture,Tables,Hon 5100 Series Wood Tables,872.940,3,0.0,157.1292,2017-07-15
1694,9830,US-2017-152842,7/16/2017,7/23/2017,Standard Class,NF-18385,Natalie Fritzler,Consumer,United States,Charlotte,...,South,FUR-CH-10004218,Furniture,Chairs,"Global Fabric Manager's Chair, Dark Gray",242.352,3,0.2,15.1470,2017-07-16


In [29]:
# year, month, week in month, day in week, holiday day, discount on day, avg discount on day

daily_df = (df.groupby('datetime').agg(total_sales=('Sales', 'sum'), avg_discount=('Discount', 'mean')).reset_index())

# Fill all days
all_days_range = pd.date_range(start=daily_df['datetime'].min(), end=daily_df['datetime'].max()) 
all_days = pd.DataFrame({'datetime': all_days_range})
# Merge with your daily data
daily_df = all_days.merge(daily_df, on='datetime', how='left')

# Replace NaN (days with no sales) by 0
daily_df['total_sales'] = daily_df['total_sales'].fillna(0)

# year
daily_df['year'] = daily_df['datetime'].dt.year

# month
daily_df['month'] = daily_df['datetime'].dt.month
daily_df['month_sin'] = np.sin(2 * np.pi * daily_df['month'] / 12)
daily_df['month_cos'] = np.cos(2 * np.pi * daily_df['month'] / 12)
daily_df.drop('month', axis = 1, inplace = True)

# week
daily_df['week_of_month'] = daily_df['datetime'].apply(lambda d: (d.day - 1) // 7 + 1)
daily_df['week_of_month_sin'] = np.sin(2 * np.pi * daily_df['week_of_month'] / 12)
daily_df['week_of_month_cos'] = np.cos(2 * np.pi * daily_df['week_of_month'] / 12)
daily_df.drop('week_of_month', axis = 1, inplace = True)

# day of week
daily_df['day_of_week'] = daily_df['datetime'].dt.dayofweek 
daily_df['day_of_week_sin'] = np.sin(2 * np.pi * daily_df['day_of_week'] / 12)
daily_df['day_of_week_cos'] = np.cos(2 * np.pi * daily_df['day_of_week'] / 12)
daily_df.drop('day_of_week', axis = 1, inplace = True)

# is discount
daily_df['is_discount'] = daily_df['avg_discount'] > 0


# holiday
us_holidays  = holidays.CountryHoliday('US', years=range(2013, 2020))
canada_holidays = holidays.CountryHoliday('CA', years=range(2013, 2020))
holiday_dates =list(canada_holidays.keys())
holiday_dates.append(list(us_holidays.keys()))
daily_df['is_holiday'] = daily_df['datetime'].dt.date.isin(holiday_dates)

# lag1
daily_df['lag1'] = daily_df['total_sales'].shift(1)

# lag7
daily_df['lag7'] = daily_df['total_sales'].shift(7)


In [30]:
daily_df

,datetime,total_sales,avg_discount,year,month_sin,month_cos,week_of_month_sin,week_of_month_cos,day_of_week_sin,day_of_week_cos,is_discount,is_holiday,lag1,lag7
0,2017-07-17,1241.1490,0.116667,2017,-5.000000e-01,-0.866025,1.000000,6.123234e-17,0.000000,1.000000e+00,True,False,NaN,NaN
1,2017-07-18,1473.8880,0.300000,2017,-5.000000e-01,-0.866025,1.000000,6.123234e-17,0.500000,8.660254e-01,True,False,1241.1490,NaN
2,2017-07-19,0.0000,NaN,2017,-5.000000e-01,-0.866025,1.000000,6.123234e-17,0.866025,5.000000e-01,False,False,1473.8880,NaN
3,2017-07-20,539.4160,0.175000,2017,-5.000000e-01,-0.866025,1.000000,6.123234e-17,1.000000,6.123234e-17,True,False,0.0000,NaN
4,2017-07-21,661.4400,0.125000,2017,-5.000000e-01,-0.866025,1.000000,6.123234e-17,0.866025,-5.000000e-01,True,False,539.4160,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,2017-12-26,0.0000,NaN,2017,-2.449294e-16,1.000000,0.866025,-5.000000e-01,0.500000,8.660254e-01,False,False,832.4540,115.378
163,2017-12-27,0.0000,NaN,2017,-2.449294e-16,1.000000,0.866025,-5.000000e-01,0.866025,5.000000e-01,False,False,0.0000,0.000
164,2017-12-28,551.2568,0.336667,2017,-2.449294e-16,1.000000,0.866025,-5.000000e-01,1.000000,6.123234e-17,True,False,0.0000,15.920
165,2017-12-29,2330.7180,0.033333,2017,-2.449294e-16,1.000000,0.500000,-8.660254e-01,0.866025,-5.000000e-01,True,False,551.2568,4086.456


In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from src.features.build_features import build_features

path = '../data/processed/train_data_processed.csv'
df = pd.read_csv('../data/train_data.csv')
df['datetime'] = pd.to_datetime(df['datetime'])

train_data_processed_df = build_features(df)
train_data_processed_df.to_csv(path,index = False)




In [26]:
path_test = '../data/processed/test_data_processed.csv'
df = pd.read_csv('../data/test_data.csv')
df['datetime'] = pd.to_datetime(df['datetime'])

data_processed_df = build_features(df)
data_processed_df.to_csv(path_test,index = False)